In [1]:
#https://stackoverflow.com/questions/12919980/nohup-is-not-writing-log-to-output-file
# nohup python -u ./gte.py > gte.log &
%load_ext autoreload
%autoreload 2
import os, sys
import time
import math
import glob

import torch
import torch.optim as O
import torch.nn as nn

from torch.autograd import Variable
from torchtext import data
from torchtext import datasets
from argparse import ArgumentParser

#from gte_netG import RNNnetG
from generator import Generator
import torch.optim as optim


In [8]:
import re
from custom_snli_loader import CustomSNLI
from enc_dec import EncDec

def tokenize(sent):
    '''
    data_reader.tokenize('a#b')
    ['a', '#', 'b']
    '''
    sent = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", sent) #.
    #if 'is is '
    return [x.strip().lower() for x in re.split('(\W+)?', sent) if x.strip()]

max_seq_len = 52#35
vocab_size = 10000
#inputs = data.Field(lower=False, fix_length=max_seq_len, init_token='<sos>')
inputs = data.Field(lower=False, fix_length=max_seq_len, tokenize=tokenize, init_token='<sos>')
answers = data.Field(sequential=False)
train, dev, test = CustomSNLI.splits(inputs, answers)
print('custom data loading debug:')
print(len(train))
print(len(dev))
print(len(test))
inputs.build_vocab(train, dev, test, max_size=vocab_size)

vector_cache = os.path.join(os.getcwd(), '.vector_cache/input_vectors.pt')
word_vectors = 'glove.42B.300d'

if os.path.isfile(vector_cache):
    inputs.vocab.vectors = torch.load(vector_cache)
else:
    #inputs.vocab.load_vectors(wv_dir=args.data_cache, wv_type=args.word_vectors, wv_dim=args.d_embed)
    print('Loading word embeddings...')
    
    inputs.vocab.load_vectors(word_vectors)
    makedirs(os.path.dirname(vector_cache))
    torch.save(inputs.vocab.vectors, vector_cache)
#answers.build_vocab(train)
    

#d_embed = 300
#n_hid = 300
#n_layers = 2
#dropout = 0.5
#rnn_type = 'GRU'
#n_embed = len(inputs.vocab)
#model = Generator(rnn_type, d_embed, n_hid, n_embed, max_seq_len)


/home/gota/.pyenv/versions/3.6.2/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


custom data loading debug:
183416
3329
3368


In [9]:
gpu = 0
#resume = '3-gru_11192017/lstm_gte_best'#'lstm_gte_e22'
#resume = '3-gru-1e-4_11202017/lstm_gte_best'
#resume = 'lstm_gte_e45'
#resume = 'lstm_gte_best.pkl'
#resume = 'saves/3-gru-attn-1e-3_11222017/lstm_gte_best'

#resume = 'luong_attn_3-gru_12052017/lstm_gte_best.pkl'
#resume = 'vnmt_pretrain_12052017/vnmt_pretrain_gru_gte_best.pkl'
#resume = 'vnmt_pretrain_1-gru_12072017/vnmt_pretrain_gru_gte_best.pkl'

resume = 'vnmt_pretrain_max52_12192017/vnmt_pretrain_gru_gte_best.pkl'
if os.path.isfile(resume):
    print("=> loading checkpoint '{}'".format(resume))
    #checkpoint = torch.load(resume)
    #start_epoch = checkpoint['epoch']
    #best_prec1 = checkpoint['best_prec1']
    #model.load_state_dict(checkpoint['state_dict'])
    #optimizer.load_state_dict(checkpoint['optimizer'])
    model = torch.load(resume, map_location=lambda storage, locatoin: storage.cuda(gpu))
    model.encoder.embeddings.weight.data = inputs.vocab.vectors
    model.decoder.embeddings.weight.data = inputs.vocab.vectors
    model.encoder.cuda()
    model.decoder.cuda()

    #print("=> loaded checkpoint '{}' (epoch {})"
    #      .format(resume, checkpoint['epoch']))
else:
    print("=> no checkpoint found at '{}'".format(resume))

=> loading checkpoint 'vnmt_pretrain_max52_12192017/vnmt_pretrain_gru_gte_best.pkl'


In [10]:
#rnn_type='LSTM'
rnn_type='GRU'
def generate(model, ntokens, example):
    """
    Generate example
    """

    #print('Generating...')
    model.encoder.eval()
    model.decoder.eval()
    SOS_TOKEN = 2
    EOS_TOKEN = 1
    out_seq = []
    
    
    # encoder initial h
    h = model.encoder.init_hidden(1) # (the very first hidden) batch_size = 1
    #print(h.data.shape)
    h.data.uniform_(-0.1, 0.1)
    
    
    inp = Variable(torch.rand(1, max_seq_len).mul(ntokens).long().cuda(), volatile=True)
    for i in range(max_seq_len):
        inp.data[0][i] = EOS_TOKEN
    for i in range(len(example)):
        inp.data[0][i] = example[i]#inputs.vocab.stoi[example[i]]
    
    #seq_lengths = torch.cuda.LongTensor( [ len(x)-list(x).count(1) for x in inp.data.cpu().numpy() ] ) # 1: <pad>
    seq_lengths = torch.cuda.LongTensor([[len(example)]])
    inp = inp.permute(1, 0)

    
    # forward encoder
    dec_type = model.dec_type
    #for i in range(len(example)):
    #    out, h = model.encoder.forward(inp[i], h, seq_lengths)
    if dec_type == 'vanilla':
        # forward encoder
        for i in range(max_seq_len):
            enc_out, h = model.encoder.forward(inp[i], h, seq_lengths)

    elif dec_type == 'attn':
        enc_outs = Variable(torch.zeros(max_seq_len, 1, model.encoder.hidden_dim)).cuda()
        #enc_outs = Variable(torch.zeros(max_seq_len, 1, model.encoder.embedding_dim)).cuda()
        for i in range(max_seq_len):
            enc_out, h = model.encoder.forward(inp[i], h, seq_lengths)
            #print(enc_out.size())

            #out, _ = pad_packed_sequence(enc_out)
            enc_outs[i] = enc_out


    # create an input with the batch_size of 1
    dec_inp = Variable(torch.LongTensor([[SOS_TOKEN]])).cuda()
    '''
    if rnn_type == 'LSTM':
        h = (h[0].cuda(), h[1].cuda())
    else:
        h = h.cuda()
    '''
    if rnn_type == 'LSTM':
        dec_h = (h[0][:model.decoder.n_layers].cuda(), h[1][:model.decoder.n_layers].cuda())
    else:
        dec_h = h[:model.decoder.n_layers].cuda()
    #print(dec_h.data.shape)


    #DECODE
    
    sample_type = 0
    for i in range(max_seq_len):
        if dec_type == 'vanilla':
            out, h = model.decoder.eval_forward(dec_inp, h, None, None)
        elif dec_type == 'attn':
            out, h, dec_attn = model.decoder.forward(dec_inp, h, enc_outs, None)
            #out, h = model.decoder.forward(dec_inp, h, enc_outs, None)

        # 0: argmax
        if sample_type == 0:
            dec_inp = out.max(1)[1]
            max_val, max_idx = out.data.squeeze().max(0)
            word_idx = max_idx[0]

        # 1: tempreture
        elif sample_type == 1:
            temperature = 1.0#1e-2
            word_weights = out.squeeze().data.div(temperature).exp().cpu()
            word_idx = torch.multinomial(word_weights, 1)[0]
            dec_inp.data.fill_(word_idx)
        
        #dec_inp = out.max(1)[1]
        output_word = inputs.vocab.itos[word_idx]
        out_seq.append(output_word)
        
        if word_idx == EOS_TOKEN:
            break
    #print(out_seq)
    return out_seq#[1:len(out_seq)-2]


def create_example(sent, N):
    example = [2]
    #example += [1] * (N - len(example)) # 1: <pad>, 2: <sos>
    words = sent.split() # e.g.'Two women and one man are drinking beer in a bar.'
    for i, w in enumerate(words):
        example.append(inputs.vocab.stoi[w])
    example.append(1)
    return example

In [5]:
ntokens = len(inputs.vocab)

sents = [
    'People are celebrating a victory on the square',
    'A wedding party looks happy on the picture',
    'Two women who just had lunch hugging and saying goodbye.',
    #'The athletes run at the start of the Haspa Hamburg marathon',
    'People are running away from the bear',
    
    'A person on a horse jumps over a broken down airplane.',
    'A boy is jumping on skateboard in the middle of a red bridge.',
    'High fashion ladies wait outside a tram beside a crowd of people in the city.',
    #'Woman in white in foreground and a man slightly behind walking with a sign for John\'s Pizza and Gyro in the background.',
    #'A woman is walking across the street eating a banana, while a man is following with his briefcase.',
    'A big brown dog swims towards the camera.',
    'A small group of church-goers watch a choir practice.',
    
    'A man is surfing in a bodysuit in beautiful blue water.',
    
    'First time a computer Go program has beaten a 9-dan professional without handicaps',
    'Competitors must be dressed in suitable tennis attire that is almost entirely white.',
    'The athletes run by the harbour during the Haspa Hamburg marathon.',
    'Space travellers gaze up and strain to find the blue dot in their sky. ',
    
    'University is designing a computer science education program to fulfill one of government \'s new education goals for the state.',
    'an airplane with skis parked in the water alongside a dock.',
    'a desk with a computer monitor and air plane model.',
    'cat looking in the mirror while the camera flash makes his eyes light up.',
    'a smiling guy sitting at a table with some food.',
    'a plate with an upside down cake on it and a bowl of fruit next to the plate.',
    'a young woman walking down the street while talking on her cell phone',
]

#model.eval()
import numpy as np
np.random.seed(1234)
for sent in sents:
    print('#'*50)
    print('#'*50)
    print(len(sent.split()))
    print(sent)

    example = create_example(sent, max_seq_len)
    print(example)
    print(generate(model, ntokens, example))
    #generate2(model, ntokens, example)
    

##################################################
##################################################
8
People are celebrating a victory on the square
[2, 0, 10, 1118, 3, 2348, 9, 5, 891, 1]
['<sos>', 'a', 'group', 'of', 'boys', 'are', 'standing', 'on', 'a', 'patio', '.', '<pad>']
##################################################
##################################################
8
A wedding party looks happy on the picture
[2, 0, 563, 456, 110, 485, 9, 5, 95, 1]
['<sos>', 'a', 'wedding', 'reception', 'with', 'a', 'picture', 'of', 'a', 'wedding', '.', '<pad>']
##################################################
##################################################
10
Two women who just had lunch hugging and saying goodbye.
[2, 0, 50, 182, 511, 1014, 1217, 762, 11, 1957, 0, 1]
['<sos>', 'a', 'woman', 'in', 'a', 'blue', 'shirt', 'hugging', 'a', 'a', 'child', 'in', 'a', 'blue', 'dress', '.', '<pad>']
##################################################
#########################################

In [11]:
import nltk

#def evaluate(val_iter, model, n_tokens, eval_batch_size, wv=None):
def evaluate_bleu(test_set=dev):
    """
    Eval acc, bleu, etc.
    """

    # Turn on evaluation mode which disables dropout.
    model.encoder.eval()
    model.decoder.eval()
    mean_bleu = 0
    skipped = 0
    mean_bleu_long=0
    count = 0
    max_seq_len=52
    
    # take mean by lengths
    interval = 5
    mean_bleu_length = [0] * int((max_seq_len-2)/interval)
    mean_bleu_count = [0] * int((max_seq_len-2)/interval)
    
    
    for pair_idx, pair in enumerate(test_set):
        #print(.premise)
        #example = [2]# 1: <pad>, 2: <sos>
        #for i, w in enumerate(pair.premise):
        #    example.append(inputs.vocab.stoi[w])
        #example.append(1)
        
        #input = ['<sos>']+pair.premise+['<pad>']
        input = pair.premise
        
        if len(input) >= max_seq_len-1:
            skipped += 1
            continue
        
        example = create_example(' '.join(input), len(input))
        prediction = generate(model, ntokens, example)
        #prediction,attn = model.generate(inputs, ntokens, example, max_seq_len)
        bleu = nltk.translate.bleu_score.sentence_bleu(pair.hypothesis, prediction)
        mean_bleu += bleu
        
        # IMPORTANT: eliminate <sos> and <pad>
        prediction = prediction[1:len(prediction)-1]
        
        
        '''print(input)
        print(example)
        print(pair.hypothesis)
        print(prediction)
        print(prediction[1:len(prediction)-1])
        print(bleu)
        fds'''
        
        
        with open('results/dev_vnmt_pretrain.txt', 'a') as the_file:
            the_file.write(' '.join(prediction) + '\n')
            
        length = len(input)
        for i in range(len(mean_bleu_length)):
            if length > i*interval and length <= (i+1)*interval:
                with open('results/dev_vnmt_pretrain_len%d.txt'%( (i+1)*interval ), 'a') as the_file:
                    the_file.write(' '.join(prediction) + '\n')
                mean_bleu_length[i] += bleu
                mean_bleu_count[i] += 1
        
        if len(pair.premise) > 20:
            mean_bleu_long += bleu
            count+=1
        
        if pair_idx % 100 == 0 and pair_idx > 0:
            print(pair_idx)
            print(mean_bleu/float(pair_idx-skipped))
            
        
    for i in range(len(mean_bleu_length)):
        if mean_bleu_count[i] != 0:
            mean_bleu_length[i] /= float(mean_bleu_count[i])
            
    print('skipped examples=%d'%skipped)
    print('mean bleus by length:')
    print(mean_bleu_length)
    print('sent counts by length:')
    print(mean_bleu_count)
    print('mean bleu for longer sentences')
    print( mean_bleu_long/float(count) )
    
    return mean_bleu/float(len(test_set)-skipped)



In [13]:
print(evaluate_bleu(dev))
print('=============')
#print(evaluate_bleu(test))


'''
test set results:
69
1. 0.670065296054072
2. (1208/2017): 0.6719217893158086

dev
80
0.6777948048325388
0.6618760696325411
'''

#80
#0.620295967183077
#0.6195247527876925
#

/home/gota/.pyenv/versions/3.6.2/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


100
0.6000603788512726
200
0.6003066418766767
300
0.5942151516058782
400
0.594156231058487
500
0.5899781960100179
600
0.5901976072292908
700
0.5870978357285349
800
0.5867832390029469
900
0.5863447330057177


/home/gota/.pyenv/versions/3.6.2/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


1000
0.5865802299385899
1100
0.585669222464475
1200
0.5872854280256209
1300
0.5873796820664001
1400
0.587413672095622
1500
0.5874716734971838
1600
0.5876663992846606
1700
0.5883838706290889
1800
0.5887993678092001
1900
0.5899065040782234
2000
0.5905557368222942
2100
0.5903879529572795
2200
0.5902135348950903
2300
0.5904366313109409
2400
0.5900921690935541
2500
0.5903931043223088
2600
0.5906591851562145
2700
0.5906358506377524
2800
0.5898203989025588
2900
0.5903247097854794
3000
0.5904842624182455
3100
0.5904191256520731
3200
0.5904865901039213
3300
0.5904963439173375
skipped examples=6
mean bleus by length:
[0.602589607950733, 0.6116464563871405, 0.5967504159740744, 0.5824045494398497, 0.5667871941835944, 0.5494018824491796, 0.5759510417686067, 0.5431502067759235, 0.4979233105752205, 0.5325062776746254]
sent counts by length:
[29, 814, 1123, 668, 448, 152, 49, 28, 5, 7]
mean bleu for longer sentences
0.5617949440631532
0.5903186839464043


'\ntest set results:\n69\n1. 0.670065296054072\n2. (1208/2017): 0.6719217893158086\n\ndev\n80\n0.6777948048325388\n0.6618760696325411\n'

In [62]:
print(len(dev))

3329


In [ ]:
0.0
0.7215811598543795 # 3-GRU w/o attention when trained w/ 1e-3
train_iter, val_iter, test_iter = data.BucketIterator.splits(
            (train, dev, test), batch_size=256, device=0)
len(train_iter)

In [18]:
from gte import create_example
def evaluate_val_pred():
    # Turn on evaluation mode which disables dropout.
    model.encoder.eval()
    model.decoder.eval()
    mean_bleu = 0
    print(max_seq_len)
    for pair_idx, pair in enumerate(dev):
        
        
        input = ['<sos>']+pair.premise+['<pad>']
        #input = pair.premise
        
        if len(input) > max_seq_len-1:
            continue
        
        if len(input) > 20:
            #print(type(input))
            #print(' '.join(input))
            example = create_example(inputs, ' '.join(input), len(input))
            #print(example)
            prediction = generate(model, ntokens, example)
            print('#'*50)
            print(input)
            print(pair.hypothesis)
            print(prediction)

        if pair_idx % 500 == 0 and pair_idx > 0:
            return

evaluate_val_pred()

35
##################################################
['<sos>', 'a', 'man', 'in', 'a', 'blue', 'shirt', 'standing', 'in', 'front', 'of', 'a', 'garage', '-', 'like', 'structure', 'painted', 'with', 'geometric', 'designs', '.', '<pad>']
['a', 'man', 'is', 'wearing', 'a', 'blue', 'shirt']
['<sos>', 'a', 'man', 'is', 'standing', 'in', 'front', 'of', 'a', 'blue', 'shirt', '.', '<pad>']
##################################################
['<sos>', 'under', 'a', 'blue', 'sky', 'with', 'white', 'clouds', ',', 'a', 'child', 'reaches', 'up', 'to', 'touch', 'the', 'propeller', 'of', 'a', 'plane', 'standing', 'parked', 'on', 'a', 'field', 'of', 'grass', '.', '<pad>']
['a', 'child', 'is', 'reaching', 'to', 'touch', 'the', 'propeller', 'of', 'a', 'plane', '.']
['<sos>', 'a', 'child', 'is', 'outside', '.', '<pad>']
##################################################
['<sos>', 'man', 'in', 'a', 'black', 'suit', ',', 'white', 'shirt', 'and', 'black', 'bowtie', 'playing', 'an', 'instrument', 'with', 'the'

['a', 'person', 'wearing', 'white', 'is', 'running', 'towards', 'a', 'ball', 'that', 'was', 'kicked', 'from', 'another', 'person', '.']
['<sos>', 'a', 'goalie', 'is', 'playing', 'a', 'game', '.', '<pad>']
##################################################
['<sos>', 'two', 'men', 'sitting', 'on', 'a', 'subway', 'are', 'reading', ',', 'with', 'coats', 'and', 'scarves', 'on', ',', 'but', 'have', 'seemed', 'to', 'have', 'lost', 'their', 'pants', '.', '<pad>']
['there', 'are', 'people', 'on', 'the', 'subway', '.']
['<sos>', 'two', 'men', 'are', 'reading', '.', '<pad>']
##################################################
['<sos>', 'a', 'man', 'with', 'a', 'beard', 'skateboarding', 'and', 'a', 'boy', 'with', 'a', 'blue', 'and', 'black', 'backpack', 'riding', 'a', 'green', 'bike', 'in', 'the', 'background', '.', '<pad>']
['there', 'is', 'a', 'man', 'and', 'a', 'boy', 'outside', '.']
['<sos>', 'a', 'man', 'is', 'riding', 'a', 'skateboard', '.', '<pad>']
##########################################

##################################################
['<sos>', 'a', 'man', 'and', 'a', 'woman', 'sit', 'on', 'a', 'deck', 'under', 'a', 'blue', 'sky', 'that', 'is', 'about', 'to', 'be', 'overtaken', 'by', 'tumultuous', 'clouds', '.', '<pad>']
['there', 'are', 'many', 'clouds', 'in', 'the', 'blue', 'sky', '.']
['<sos>', 'a', 'man', 'and', 'woman', 'are', 'sitting', 'on', 'a', 'deck', '.', '<pad>']
##################################################
['<sos>', 'two', 'women', 'stand', 'on', 'a', 'beach', 'and', 'perform', 'stretches', 'or', 'yoga', 'together', ',', 'facing', 'towards', 'the', 'water', 'and', 'away', 'from', 'the', 'camera', '.', '<pad>']
['two', 'people', 'are', 'exercising', 'outside', '.']
['<sos>', 'two', 'women', 'are', 'standing', 'on', 'the', 'beach', '.', '<pad>']
##################################################
['<sos>', 'people', 'in', 'a', 'grocery', 'store', 'crowd', 'around', 'to', 'watch', 'a', 'tv', ',', 'except', 'one', 'man', 'who', 'walks', 'toward', 'the'

##################################################
['<sos>', 'a', 'woman', 'wearing', 'gray', 'pants', ',', 'a', 'white', 'blouse', 'and', 'a', 'black', 'vest', 'is', 'jumping', 'with', 'one', 'hand', 'in', 'the', 'air', 'as', 'she', 'goes', 'through', 'an', 'indoor', 'stadium', '.', '<pad>']
['the', 'jumping', 'lady', 'in', 'slacks', 'also', 'has', 'her', 'hand', 'raised', '.']
['<sos>', 'a', 'woman', 'is', 'jumping', '.', '<pad>']
##################################################
['<sos>', 'a', 'brunette', 'woman', 'in', 'a', 'multicolored', 'floral', 'outfit', 'and', 'a', 'pearl', 'necklace', 'is', 'talking', 'while', 'holding', 'a', 'wineglass', '.', '<pad>']
['a', 'woman', 'holds', 'a', 'glass']
['<sos>', 'a', 'woman', 'is', 'talking', 'to', 'a', '.', '<pad>']
##################################################
['<sos>', 'a', 'male', 'soccer', 'player', 'has', 'his', 'foot', 'extended', 'as', 'if', 'he', 'has', 'just', 'kicked', 'the', 'soccer', 'ball', 'while', 'the', 'ball', 'fl

##################################################
['<sos>', 'a', 'guy', 'with', 'long', 'brown', 'curly', 'hair', 'wearing', 'a', 'white', 't', '-', 'shirt', 'is', 'looking', 'into', 'the', 'camera', 'with', 'something', 'in', 'his', 'mouth', '.', 'a', '<pad>']
['a', 'person', 'with', 'long', 'hair', 'looks', 'at', 'the', 'camera']
['<sos>', 'a', 'man', 'is', 'looking', 'into', 'a', 'camera', '.', '<pad>']
##################################################
['<sos>', 'a', 'woman', 'is', 'holding', 'one', 'young', 'children', 'with', 'another', 'standing', 'next', 'to', 'her', 'while', 'a', 'man', 'is', 'sitting', 'by', 'a', 'small', 'campfire', '.', '<pad>']
['a', 'woman', 'is', 'holding', 'one', 'young', 'children', 'with', 'another', 'standing', 'next', 'to', 'her']
['<sos>', 'a', 'woman', 'is', 'holding', 'a', 'baby', '.', '<pad>']
##################################################
['<sos>', 'a', 'basset', 'hound', 'is', 'tied', 'to', 'a', 'doorway', 'in', 'an', 'alley', 'in', 'fron

##################################################
['<sos>', 'the', 'front', 'of', 'the', 'cafe', 'must', 'remain', 'free', 'of', 'trash', 'if', 'customers', 'are', 'to', 'be', 'enticed', 'to', 'enter', '.', '<pad>']
['the', 'front', 'door', 'is', 'clear', 'of', 'trash', '.']
['<sos>', 'the', 'market', 'is', 'full', 'of', 'items', '.', '<pad>']
##################################################
['<sos>', 'a', 'woman', 'with', 'short', 'blond', '-', 'hair', 'smiling', 'and', 'singing', 'into', 'a', 'microphone', 'while', 'a', 'man', 'in', 'a', 'striped', 'shirt', ',', 'further', 'back', ',', 'plays', 'an', 'acoustic', 'guitar', '.', '<pad>']
['two', 'people', 'are', 'performing', '.']
['<sos>', 'a', 'woman', 'is', 'singing', '.', '<pad>']
##################################################
['<sos>', 'a', 'person', 'on', 'a', 'bicycle', 'wears', 'a', 'helmet', 'and', 'jumps', ',', 'soaring', 'over', 'the', 'ground', 'in', 'front', 'of', 'a', 'skyline', 'and', 'lake', '.', '<pad>']
['a', '